In [11]:
import librosa
import numpy as np
import os
import glob
from sklearn.preprocessing import MinMaxScaler
max_value = float('-inf')  

input_dir = './train'  # 컴파일시 test와 train 디렉토리 두개 다 전처리 해야함
output_dir = './CQT_mixed_train'  # 결과를 저장할 디렉토리 경로, test 전처리 진행시 './CQT_test'로 진행

os.makedirs(output_dir, exist_ok=True)
ogg_files = glob.glob(os.path.join(input_dir, '*.wav'))
target_frames = 400

In [ ]:
for file_path in ogg_files:
    file_name = os.path.splitext(os.path.basename(file_path))[0]
    y, sr = librosa.load(file_path, sr=None)  

    # CQT 파라미터 설정
    hop_length = int(sr * 0.016)  # 스텝 사이즈: 16ms마다 프레임 이동
    n_bins_per_octave = 36       # 옥타브당 빈 개수
    n_octaves = 7                # 옥타브 수
    n_bins = n_bins_per_octave * n_octaves  # 전체 빈 수
    
    cqt_db = np.abs(librosa.cqt(y, sr=sr, hop_length=hop_length, n_bins=n_bins, bins_per_octave=n_bins_per_octave))
    if max_value < cqt_db.max():
        max_value = cqt_db.max()
    n_frames = cqt_db.shape[1]
    
    if n_frames > target_frames: 
        # 프레임이 많을 경우, 중앙 부분을 자름
        start = (n_frames - target_frames) // 2
        cqt_db = cqt_db[:, start:start + target_frames]
    elif n_frames < target_frames:
        # 프레임이 부족할 경우, 프레임을 반복하여 확장함
        repeats = target_frames // n_frames
        remainder = target_frames % n_frames
        cqt_db = np.tile(cqt_db, (1, repeats)) 
        cqt_db = np.concatenate((cqt_db, cqt_db[:, :remainder]), axis=1)  # 부족한 부분을 채우기 위해 나머지를 추가

    


    
    # 결과 저장 
    output_path = os.path.join(output_dir, f'{file_name}.npy')
    np.save(output_path, cqt_db)
    


In [7]:
max_value

38.85052

max_value 값은 38.85052 (이건 train 데이터 최댓값)

In [13]:
for file_path in ogg_files:
    file_name = os.path.splitext(os.path.basename(file_path))[0]
    y, sr = librosa.load(file_path, sr=None) 
    
    # CQT 파라미터 설정
    hop_length = int(sr * 0.016)  # 스텝 사이즈: 16ms마다 프레임 이동
    n_bins_per_octave = 36       # 옥타브당 빈 개수
    n_octaves = 7                # 옥타브 수
    n_bins = n_bins_per_octave * n_octaves  # 전체 빈 수
    
    cqt_db = np.abs(librosa.cqt(y, sr=sr, hop_length=hop_length, n_bins=n_bins, bins_per_octave=n_bins_per_octave))

    n_frames = cqt_db.shape[1]
    if n_frames > target_frames:
        # 프레임이 많을 경우, 중앙 부분을 자릅니다.
        start = (n_frames - target_frames) // 2
        cqt_db = cqt_db[:, start:start + target_frames]
    elif n_frames < target_frames:
        # 프레임이 부족할 경우, 프레임을 반복하여 확장합니다.
        repeats = target_frames // n_frames
        remainder = target_frames % n_frames
        cqt_db = np.tile(cqt_db, (1, repeats))  
        cqt_db = np.concatenate((cqt_db, cqt_db[:, :remainder]), axis=1)  # 부족한 부분을 채우기 위해 나머지를 추가
    


    # minmax
    min_val = 0
    max_val = 38.85052
    scaled_cqt = (cqt_db - min_val) / (max_val - min_val)

    # 결과 저장
    output_path = os.path.join(output_dir, f'{file_name}.npy')
    np.save(output_path, scaled_cqt)

 
   
